# Preparing tools

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os
import re
import string
from random import randint

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Dropout, GRU


# Reading text file

In [2]:
def read_txt(filename):
    file = open(filename, 'r', encoding="utf8")
    text = file.read()
    file.close()
    return text

In [3]:
in_file = 'pg1497.txt'
txt = read_txt(in_file)
print(txt[:800])

﻿The Project Gutenberg eBook of The Republic, by Plato

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: The Republic

Author: Plato

Translator: B. Jowett

Release Date: October, 1998 [eBook #1497]
[Most recently updated: September 11, 2021]

Language: English


Produced by: Sue Asscher and David Widger

*** START OF THE PROJECT GUTENBERG EBOOK THE REPUBLIC ***




THE REPUBLIC

By Plato

Trans


In [4]:
[m.start() for m in re.finditer('BOOK I\.', txt)]

[967, 38188, 553671]

In [5]:
[m.start() for m in re.finditer('End of', txt)]

[]

In [6]:
books = txt[553615:1195644]

print(books[:300])

ameless person, who are
introduced in the Timaeus.




 BOOK I.


I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what manner they would
celebrate the festival, 


### Text cleaning

In [7]:
def remove_punctuation(text):
    PUNCT_TO_REMOVE = string.punctuation
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [8]:
cleaned = remove_punctuation(books)

tokens = cleaned.split()
tokens = [word for word in tokens if word.isalpha()]
tokens = [word.lower() for word in tokens]

print('Tokens :\n',tokens[:10])
print('No of Tokens : ',len(tokens))
print('Unique Tokens : ',len(set(tokens)))

Tokens :
 ['ameless', 'person', 'who', 'are', 'introduced', 'in', 'the', 'timaeus', 'book', 'i']
No of Tokens :  117417
Unique Tokens :  7339


In [9]:
sequences = list()
for i in range(0,len(tokens),51):
    seq = tokens[i:i+51]
    line = ' '.join(seq)
    sequences.append(line)
    
print('Number of Sequences : ',len(sequences))

Number of Sequences :  2303


### Encoding

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
encoded = tokenizer.texts_to_sequences(sequences)
encoded = np.array(encoded[:-1])

In [11]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

7340

#### Spliting labels from our data

In [12]:
X , y = encoded[:,:-1], encoded[:,-1]

y = to_categorical(y, num_classes=vocab_size)

seq_length = X.shape[1]

print( X.shape)
print( y.shape)

(2302, 50)
(2302, 7340)


## Building our model


In [13]:
LSTM_model = Sequential()

LSTM_model.add(Embedding(vocab_size, 100, input_length=seq_length))
LSTM_model.add(LSTM(100, return_sequences=True,dropout=0.1))
LSTM_model.add(LSTM(100, return_sequences=True,dropout=0.1))
LSTM_model.add(LSTM(80))
LSTM_model.add(Dense(300))
LSTM_model.add(Dropout(0.2))
LSTM_model.add(Dense(vocab_size, activation='softmax'))


    

In [14]:
LSTM_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
LSTM_model.summary()
  

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           734000    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           80400     
                                                                 
 lstm_1 (LSTM)               (None, 50, 100)           80400     
                                                                 
 lstm_2 (LSTM)               (None, 80)                57920     
                                                                 
 dense (Dense)               (None, 300)               24300     
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 7340)              2

### Training Model

In [15]:
LSTM_model.fit(X, y, batch_size=128, epochs=150)

Epoch 1/150
18/18 [==============================] - 7s 169ms/step - loss: 8.1573 - accuracy: 0.0547
Epoch 2/150
18/18 [==============================] - 3s 160ms/step - loss: 6.0702 - accuracy: 0.0530
Epoch 3/150
18/18 [==============================] - 3s 168ms/step - loss: 5.7822 - accuracy: 0.0556
Epoch 4/150
18/18 [==============================] - 3s 166ms/step - loss: 5.6720 - accuracy: 0.0482
Epoch 5/150
18/18 [==============================] - 3s 172ms/step - loss: 5.6315 - accuracy: 0.0582
Epoch 6/150
18/18 [==============================] - 3s 170ms/step - loss: 5.6322 - accuracy: 0.0599
Epoch 7/150
18/18 [==============================] - 3s 175ms/step - loss: 5.6220 - accuracy: 0.0573
Epoch 8/150
18/18 [==============================] - 3s 169ms/step - loss: 5.6162 - accuracy: 0.0599
Epoch 9/150
18/18 [==============================] - 3s 169ms/step - loss: 5.6184 - accuracy: 0.0547
Epoch 10/150
18/18 [==============================] - 3s 179ms/step - loss: 5.6148 - accura

### Generating Text

In [16]:
def generate_txt(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        yhat = np.argmax(model.predict(encoded),axis=1)

        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [17]:
seed_text = sequences[randint(0,len(sequences))]
print(seed_text + '\n')

new = generate_txt(LSTM_model, tokenizer, seq_length, seed_text, 50) 
print(new)

like a god maintains justice to whom the black earth brings forth wheat and barley whose trees are bowed with fruit and his sheep never fail to bear and the sea gives him still grander are the gifts of heaven which musaeus and his son vouchsafe to the just they take

same only mistaken of fain not refrain to enemies not yourself nearly business unjust for sort he together to wish which there be are more the other instrument although but his state but the up for worthless the other falls but he nay the defence which what if what if


### GRU

#### Building the model

In [18]:

GRU_model = Sequential()
GRU_model.add(Embedding(vocab_size, 100, input_length=seq_length))
GRU_model.add(GRU(100, return_sequences=True,dropout=0.1))
GRU_model.add(GRU(100, return_sequences=True,dropout=0.1))
GRU_model.add(GRU(80))
GRU_model.add(Dense(300))
GRU_model.add(Dropout(0.2))
GRU_model.add(Dense(vocab_size, activation='softmax'))

   

In [19]:

GRU_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
GRU_model.summary()
   

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 100)           734000    
                                                                 
 gru (GRU)                   (None, 50, 100)           60600     
                                                                 
 gru_1 (GRU)                 (None, 50, 100)           60600     
                                                                 
 gru_2 (GRU)                 (None, 80)                43680     
                                                                 
 dense_2 (Dense)             (None, 300)               24300     
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense_3 (Dense)             (None, 7340)             

#### Training

In [20]:
GRU_model.fit(X, y, batch_size=128, epochs=150)

Epoch 1/150
18/18 [==============================] - 6s 132ms/step - loss: 8.2962 - accuracy: 0.0547
Epoch 2/150
18/18 [==============================] - 2s 133ms/step - loss: 6.0796 - accuracy: 0.0504
Epoch 3/150
18/18 [==============================] - 2s 139ms/step - loss: 5.7450 - accuracy: 0.0517
Epoch 4/150
18/18 [==============================] - 3s 141ms/step - loss: 5.6655 - accuracy: 0.0578
Epoch 5/150
18/18 [==============================] - 3s 141ms/step - loss: 5.6371 - accuracy: 0.0534
Epoch 6/150
18/18 [==============================] - 3s 140ms/step - loss: 5.6185 - accuracy: 0.0569
Epoch 7/150
18/18 [==============================] - 3s 140ms/step - loss: 5.6149 - accuracy: 0.0595
Epoch 8/150
18/18 [==============================] - 3s 140ms/step - loss: 5.6079 - accuracy: 0.0526
Epoch 9/150
18/18 [==============================] - 3s 141ms/step - loss: 5.5707 - accuracy: 0.0539
Epoch 10/150
18/18 [==============================] - 3s 140ms/step - loss: 5.4891 - accura

In [22]:
predict = generate_txt(GRU_model, tokenizer, seq_length, seed_text, 50) 
print(predict)

take take image anybody anybody trusted trusted count count could count count count count distress distress distress accordingly geometrician distress distress distress geometrician pursued pursued accordingly geometrician note note note note make explain arrive pantomimic tyrant tyrant lusts tyrant dead perverse rules dead dead perverse question dead perverse elevating elevating


## Reference

https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/#:~:text=Language%20Model%20Design&text=The%20language%20model%20will%20be,the%20input%20sequences%20should%20be.
https://www.kaggle.com/code/ashrafkhan94/the-republic-by-plato-nl-model-text-generation
